In [46]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
import os

# Load variables from .env into environment
load_dotenv()

# Initialize model with API key and base URL from .env
model = ChatOpenAI(
    model="text-embedding-3-small",
    api_key=os.getenv("OPEN_API_KEY"),        # match variable name in .env
    openai_api_base=os.getenv("OPENAI_BASE_URL")
)

## Install libraries

In [18]:
# !pip install -q youtube-transcript-api langchain-community langchain-openai \
#                faiss-cpu tiktoken python-dotenv

In [27]:
!pip install pytubefix


   ---------------------------------------- 0.0/768.9 kB ? eta -:--:--
   --------------------------- ------------ 524.3/768.9 kB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 768.9/768.9 kB 2.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import youtube_transcript_api
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

## Step 1a - Indexing (Document Ingestion)

In [35]:
from pytubefix import YouTube
from xml.etree import ElementTree as ET

video_id = "Gfr50f6ZBvo"  # only the ID, not full URL
try:
    yt = YouTube(f"https://www.youtube.com/watch?v={video_id}")

    # If you don’t care which language, this returns the “best” one
    preferred_codes = ["en.eEY6OEpapPo", "a.en", "en"]
    transcript_list = None

    for code in preferred_codes:
        if code in yt.captions:
            xml_captions = yt.captions[code].xml_captions
            root = ET.fromstring(xml_captions)
            transcript_list = [{"text": el.text.strip()} for el in root.findall(".//text") if el.text]
            break

    if transcript_list:
        # Flatten it to plain text
        transcript = " ".join(chunk["text"] for chunk in transcript_list)
        print(transcript)
    else:
        print("No captions available for this video.")

except Exception:
    print("No captions available for this video.")


the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i&#39;m sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here&#39;s demis hassabis let&#39;s start with a bit of a personal question am i an ai program you wrote to interview people until i get 

In [36]:
transcript_list

[{'text': 'the following is a conversation with'},
 {'text': 'demus hasabis'},
 {'text': 'ceo and co-founder of deepmind'},
 {'text': 'a company that has published and builds'},
 {'text': 'some of the most incredible artificial'},
 {'text': 'intelligence systems in the history of'},
 {'text': 'computing including alfred zero that'},
 {'text': 'learned'},
 {'text': 'all by itself to play the game of gold'},
 {'text': 'better than any human in the world and'},
 {'text': 'alpha fold two that solved protein'},
 {'text': 'folding'},
 {'text': 'both tasks considered nearly impossible'},
 {'text': 'for a very long time'},
 {'text': 'demus is widely considered to be one of'},
 {'text': 'the most brilliant and impactful humans'},
 {'text': 'in the history of artificial'},
 {'text': 'intelligence and science and engineering'},
 {'text': 'in general'},
 {'text': 'this was truly an honor and a pleasure'},
 {'text': 'for me to finally sit down with him for'},
 {'text': 'this conversation and i&#39;

## Step 1b - Indexing (Text Splitting)

In [50]:
splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [51]:
len(chunks)

274

In [52]:
chunks[100]

Document(metadata={}, page_content='specifically trained to only play go right so uh and what we wanted to do with first version of go is just get to world champion performance no matter how we did it right and then and then of course alphago zero we we we removed the need to use human games as a starting point right so it could just play against itself from random starting point from the beginning so that removed the the need for human knowledge uh about go and then finally alpha zero then generalized it so that any things we had in there the system including things like symmetry of the go board uh were removed so the alpha zero could play from scratch any two player game and then mu0 which is the final latest version of')

## Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [53]:

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=os.getenv("OPEN_API_KEY"),
    openai_api_base=os.getenv("OPENAI_BASE_URL")
)
vector_store = FAISS.from_documents(chunks, embeddings)

UnprocessableEntityError: Error code: 422 - {'detail': [{'type': 'string_type', 'loc': ['body', 'input', 'str'], 'msg': 'Input should be a valid string', 'input': [[1820, 2768, 374, 264, 10652, 449, 2486, 355, 706, 15728, 3846, 78, 323, 1080, 36927, 315, 5655, 38824, 264, 2883, 430, 706, 4756, 323, 22890, 1063, 315, 279, 1455, 15400, 21075, 11478, 6067, 304, 279, 3925, 315, 25213, 2737, 453, 29093, 7315, 430, 9687, 682, 555, 5196, 311, 1514, 279, 1847, 315, 6761, 2731, 1109, 904, 3823, 304, 279, 1917, 323, 8451, 11816, 1403, 430, 29056, 13128, 45842, 2225, 9256, 6646, 7154, 12266, 369, 264, 1633, 1317, 892, 2486, 355, 374, 13882, 6646, 311, 387, 832, 315, 279, 1455, 20333, 323, 98990, 12966, 304, 279, 3925, 315, 21075, 11478, 323, 8198, 323, 15009, 304, 4689, 420, 574, 9615, 459, 16044, 323, 264, 17069, 369, 757, 311, 5616, 2503, 1523, 449, 1461, 369, 420, 10652, 323, 602, 28977, 2137, 68336], [28400, 598, 304, 279, 3925, 315, 21075, 11478, 323, 8198, 323, 15009, 304, 4689, 420, 574, 9

In [41]:
vector_store.index_to_docstore_id

NameError: name 'vector_store' is not defined

In [42]:
vector_store.get_by_ids(['2436bdb8-3f5f-49c6-8915-0c654c888700'])

NameError: name 'vector_store' is not defined

## Step 2 - Retrieval

In [43]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

NameError: name 'vector_store' is not defined

In [44]:
retriever

NameError: name 'retriever' is not defined

In [45]:
retriever.invoke('What is deepmind')

NameError: name 'retriever' is not defined

## Step 3 - Augmentation

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

In [ ]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [ ]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [ ]:
retrieved_docs

[Document(id='3c60d0d6-5d01-4dfc-99fc-5c4bb4422cb0', metadata={}, page_content="so we with this problem and we published it in a nature paper last year uh we held the fusion that we held the plasma in specific shapes so actually it's almost like carving the plasma into different shapes and control and hold it there for the record amount of time so um so that's one of the problems of of fusion sort of um solved so i have a controller that's able to no matter the shape uh contain it continue yeah contain it and hold it in structure and there's different shapes that are better for for the energy productions called droplets and and and so on so um so that was huge and now we're looking we're talking to lots of fusion startups to see what's the next problem we can tackle uh in the fusion area so another fascinating place in a paper title pushing the frontiers of density functionals by solving the fractional electron problem so you're taking on modeling and simulating the quantum mechanical 

In [ ]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"so we with this problem and we published it in a nature paper last year uh we held the fusion that we held the plasma in specific shapes so actually it's almost like carving the plasma into different shapes and control and hold it there for the record amount of time so um so that's one of the problems of of fusion sort of um solved so i have a controller that's able to no matter the shape uh contain it continue yeah contain it and hold it in structure and there's different shapes that are better for for the energy productions called droplets and and and so on so um so that was huge and now we're looking we're talking to lots of fusion startups to see what's the next problem we can tackle uh in the fusion area so another fascinating place in a paper title pushing the frontiers of density functionals by solving the fractional electron problem so you're taking on modeling and simulating the quantum mechanical behavior of electrons yes um can you explain this work and can ai model and\n\n

In [ ]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [ ]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      so we with this problem and we published it in a nature paper last year uh we held the fusion that we held the plasma in specific shapes so actually it's almost like carving the plasma into different shapes and control and hold it there for the record amount of time so um so that's one of the problems of of fusion sort of um solved so i have a controller that's able to no matter the shape uh contain it continue yeah contain it and hold it in structure and there's different shapes that are better for for the energy productions called droplets and and and so on so um so that was huge and now we're looking we're talking to lots of fusion startups to see what's the next problem we can tackle uh in the fusion area so another fascinating place in a paper title pushing the frontiers of density functionals

## Step 4 - Generation

In [ ]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of nuclear fusion is discussed in the video. The discussion includes the following points:

1. The speaker mentions a problem in fusion that was published in a Nature paper, where they developed a controller that can hold plasma in specific shapes for a record amount of time, which is crucial for energy production.

2. They talk about collaborating with EPFL in Switzerland, which has a test reactor that they used for their experiments. The focus is on identifying bottleneck problems in fusion and applying AI methods to address those challenges.

3. The speaker emphasizes the potential of AI to help accelerate solutions in energy and climate, specifically mentioning fusion as an area where AI can contribute.

4. They also mention their work on magnetic control of tokamak plasmas using deep reinforcement learning, indicating that they are exploring how AI can assist in controlling high-temperature plasmas for nuclear fusion.


## Building a Chain

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [ ]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [ ]:
parallel_chain.invoke('who is Demis')

{'context': "the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get

In [ ]:
parser = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('Can you summarize the video')

'The video features a conversation with Demas, who discusses the need for deeper and simpler explanations in physics, particularly in relation to consciousness, life, and gravity. He emphasizes the limitations of the current standard model of physics and the importance of exploring more fundamental explanations. Additionally, he talks about advancements in fusion research, specifically how they have managed to hold plasma in specific shapes for extended periods, which is a significant step in fusion energy production. The conversation touches on the potential of AI in modeling quantum mechanical behavior, particularly regarding electrons. The discussion concludes with a quote about the nature of computer science.'